## Importation et modification données

In [1]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""


'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [3]:
from sklearn.preprocessing import MinMaxScaler

def normalisation(data):
    columns = ["track_popularity","danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo", "duration_ms", "track_album_release_year"]
    data_norme = data[columns]
    scaler = MinMaxScaler(feature_range=(0,1))
    data_norme = pd.DataFrame(scaler.fit_transform(data_norme), columns=columns)
    return data_norme


In [4]:
def dedoublons(data):
    return data.drop_duplicates(subset ="track_name",keep = False, inplace=True)
    
dedoublons(spotify_data_cleaned)
spotify_data_cleaned

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_month,track_album_release_year
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093,07,2019
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052,03,2019
6,2OAylPUDDfwRGfe0lYqlCQ,Never Really Over - R3HAB Remix,Katy Perry,62,7INHYSeusaFlyrHSNxm8qH,Never Really Over (R3HAB Remix),Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,0,0.0623,0.187000,0.000000,0.1760,0.1520,112.648,187675,07,2019
10,3eFJI470ALf1EYjywPRsJy,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58,49bXZtlI5kQSqQMSCnSaWO,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.1810,0.146000,0.000005,0.1240,0.7520,121.984,207894,06,2019
12,6e0CvGZf7CouOpYF8toXHC,Body On My,Loud Luxury,67,2R66I0nzp3NnuUlzIGWQ5P,Body On My,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,...,1,0.0463,0.039900,0.000000,0.3740,0.6870,121.985,192507,03,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375,04,2014
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120,03,2013
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112,04,2014
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,...,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432,01,2014


## KNN

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

**Fonction pour appliquer la KNN**

In [6]:
def KNN(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, random_state=0)

    n_neighbors_list = np.unique(np.round(np.geomspace(1, 500, 100)).astype(int))
    param_grid = {"n_neighbors": n_neighbors_list}

    cls = KNeighborsClassifier()

    search = GridSearchCV(cls, param_grid, scoring="accuracy", cv=10)
    search.fit(xx, yy)
    kopt = search.best_params_
    print(kopt)
    
    knn = KNeighborsClassifier(n_neighbors = kopt["n_neighbors"])
    #training
    knn.fit(x_train,y_train)
    print(knn.score(x_test,y_test))

**Fonction pour appliquer KNN sur 2 genres**

In [7]:
def KNN_2_genre(genre1,genre2):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2

    data_1= spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    
    data= pd.concat([data_1,data_2])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 3 genres**

In [8]:
def KNN_3_genre(genre1,genre2,genre3):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3

    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]

    data= pd.concat([data_1,data_2,data_3])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 4 genres**

In [9]:
def KNN_4_genre(genre1,genre2,genre3,genre4):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3
    mask_4 = spotify_data_cleaned.playlist_genre == genre4


    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]
    data_4 = spotify_data_cleaned.loc[mask_4]

    data= pd.concat([data_1,data_2,data_3,data_4])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Application sur tous les cas**

In [10]:
print("EDM - RAP")
KNN_2_genre("edm","rap")
print("---------------------")

print("EDM - ROCK")
KNN_2_genre("edm","rock")
print("---------------------")

print("EDM - R&B")
KNN_2_genre("edm","r&b")
print("---------------------")

print("EDM - latin")
KNN_2_genre("edm","latin")
print("---------------------")

print("EDM - pop")
KNN_2_genre("edm","pop")
print("---------------------")

print("RAP - ROCK")
KNN_2_genre("rap","rock")
print("---------------------")

print("RAP - R&B")
KNN_2_genre("rap","r&b")
print("---------------------")

print("RAP - LATIN")
KNN_2_genre("rap","latin")
print("---------------------")

print("rap - pop")
KNN_2_genre("rap","pop")
print("---------------------")

print("rock - r&b")
KNN_2_genre("rock","r&b")
print("---------------------")

print("rock - latin")
KNN_2_genre("rock","latin")
print("---------------------")

print("rock - pop")
KNN_2_genre("rock","pop")
print("---------------------")

print("r&b - latin")
KNN_2_genre("r&b","latin")
print("---------------------")

print("r&b - pop")
KNN_2_genre("r&b","pop")
print("---------------------")

print("latin - pop")
KNN_2_genre("latin","pop")


EDM - RAP
{'n_neighbors': 11}
0.8558951965065502
---------------------
EDM - ROCK
{'n_neighbors': 9}
0.8986083499005965
---------------------
EDM - R&B
{'n_neighbors': 19}
0.8672120830711139
---------------------
EDM - latin
{'n_neighbors': 49}
0.8048780487804879
---------------------
EDM - pop
{'n_neighbors': 23}
0.7537655533726261
---------------------
RAP - ROCK
{'n_neighbors': 11}
0.9342665855143031
---------------------
RAP - R&B
{'n_neighbors': 49}
0.7514518002322881
---------------------
RAP - LATIN
{'n_neighbors': 11}
0.766548463356974
---------------------
rap - pop
{'n_neighbors': 25}
0.8217940999397954
---------------------
rock - r&b
{'n_neighbors': 9}
0.8327376697641172
---------------------
rock - latin
{'n_neighbors': 9}
0.8568298027757487
---------------------
rock - pop
{'n_neighbors': 7}
0.8191330343796711
---------------------
r&b - latin
{'n_neighbors': 111}
0.7030386740331491
---------------------
r&b - pop
{'n_neighbors': 67}
0.7353563867325336
-------------------

**Application sur les trois genres qui semblent le plus facile à prédire**

In [11]:
KNN_3_genre("edm",'rap','rock')

{'n_neighbors': 17}
0.8013643659711075


**Tentative sur 4 genres**


In [12]:
KNN_4_genre("edm",'rap','rock','latin')

{'n_neighbors': 30}
0.6790625


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>